# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [9]:
#Your code here
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

ha = pd.read_csv('homepage_actions.csv')
print(ha.info())
ha.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
ha['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [6]:
clickers = set(ha[ha['action']=='click']['id'].unique())
viewers = set(ha[ha['action']=='view']['id'].unique())
print('The website had {} viewers of which {} clicked.'.format(len(viewers),
                                                               len(clickers)))
print('{} viewed the website without clicking.'.format(len(viewers-clickers)))
print('{} clicked the website without viewing.'.format(len(clickers-viewers)))

The website had 6328 viewers of which 1860 clicked.
4468 viewed the website without clicking.
0 clicked the website without viewing.


In [16]:
groups = ['control','experiment']
ids = []
for group in groups:
    ids.append(set(ha[ha.group==group]['id'].unique()))
print('There were {} overlaps between {} and {} groups'.format(len(ids[0]&ids[1]),
                                                               groups[0],
                                                               groups[1]))

There were 0 overlaps between control and experiment groups


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [26]:
#Your code here
control = ha[ha.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = ha[ha.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), 
                                                          len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), 
                                                          experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), 
                                                                experiment.click.mean()))

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


In [27]:
from flatiron_stats import *

p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [32]:
#Your code here
control_rate = control.click.mean()
expected_experiment_clicks = control_rate * len(experiment)
print(expected_experiment_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [29]:
#Your code here
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [33]:
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [34]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
'''
The p-values calculated through the Welch's test and using z-score have some 
offset but both are significantly less than alpha = 0.01. Therefore, we can 
reject the null hypothesis, in this case, it was 'There is not statistical 
difference between the experimental and control groups.'  
''' 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.